## Installing dependencies

In [1]:
!pip install selenium

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.5 MB 9.6 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.5 MB 9.6 MB/s eta 0:00:01
   -------------------- ------------------- 5.0/9.5 MB 8.9 MB/s eta 0:00:01
   ---------------------------- ----------- 6.8/9.5 MB 8.9 MB/s eta 0:00:01
   ------------------------------------ --- 8.7/9.5 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 8.4 MB/s eta 0:00:00
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-non

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
httpx 0.27.2 requires anyio, which is not installed.
langsmith 0.1.136 requires requests<3,>=2, which is not installed.
langsmith 0.1.136 requires requests-toolbelt<2.0.0,>=1.0.0, which is not installed.


### importing selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service

###Other imports here

In [3]:
%pip install webdriver_manager
%pip install wget
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/12.6 MB 9.4 MB/s eta 0:00:02
   ----------- ---------------------------- 3.7/12.6 MB 9.9 MB/s eta 0:00:01
   ------------------ --------------------- 5.8/12.6 MB 9.8 MB/s eta 0:00:01
   ------------------------ --------------- 7.9/12.6 MB 9.7 MB/s eta 0:00:01
   ------------------------------ --------- 9.7/12.6 MB 9.7 MB/s eta 0:00:01
   ------------------------------------ --- 11.5/12.6 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.6 MB 9.1 MB/s eta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fitz 0.0.1.dev2 requires scipy, which is not installed.
langchain 0.3.4 requires aiohttp<4.0.0,>=3.8.3, which is not installed.
langchain 0.3.4 requires PyYAML>=5.3, which is not installed.
langchain 0.3.4 requires SQLAlchemy<3,>=1.4, which is not installed.
langchain 0.3.4 requires tenacity!=8.4.0,<10,>=8.1.0, which is not installed.
librosa 0.10.2.post1 requires joblib>=0.14, which is not installed.
librosa 0.10.2.post1 requires lazy-loader>=0.1, which is not installed.
librosa 0.10.2.post1 requires msgpack>=1.0, which is not installed.
librosa 0.10.2.post1 requires numba>=0.51.0, which is not installed.
librosa 0.10.2.post1 requires scikit-learn>=0.20.0, which is not installed.
librosa 0.10.2.post1 requires scipy>=1.2.0, which is not installed.
mlxtend 0.23.4 requires joblib>=0.13.2, which is not installed.
mlx

In [4]:
import os
import wget
import time
import json
from urllib.parse import urljoin
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

chromeDriver Path


In [11]:
chrome_driver_path = r"C:\Users\Chetan\chromedriver-win32\chromedriver.exe"

In [ ]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

missions = ["insat-3d", "insat-3dr","megha-tropiques", "scatsat-1", "kalpana-1", "saral-altika", "oceansat-2", "oceansat-3"]
mission_subpages = ["introduction", "objectives", "spacecraft", "payloads", "references"]

# Base URL
base_url = "https://www.mosdac.gov.in"
pdf_icon_url = "https://www.mosdac.gov.in/images/PDF.gif"

# Dictionary to store extracted data
mission_data_store = {}

# for _ in range(1):
for mission in missions:
    mission_url = f"{base_url}/{mission}"
    driver.get(mission_url)
    # driver.get("https://www.mosdac.gov.in/"+missions[6])
    
    try:
        # Wait for the content area to load
        content_area = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".field-item.even"))
        )
        print(f"Element found for: {mission}")

        # Extract data from the main mission page
        mission_data = {
            "text": content_area.text,
            "images": [],
            "tables": [],
            "pdfs": []
        }

        # Extract all images within the section
        images = content_area.find_elements(By.TAG_NAME, "img")
        for img in images:
            img_src = urljoin(base_url, img.get_attribute("src"))
            if img_src != pdf_icon_url:
                mission_data["images"].append(img_src)

        # Extract all tables
        tables = content_area.find_elements(By.TAG_NAME, "table")
        for table in tables:
            mission_data["tables"].append(table.get_attribute("outerHTML"))  # table parsing will be done later

        # Extract all PDF links
        pdf_links = content_area.find_elements(By.CSS_SELECTOR, "p a[href$='.pdf']")
        for pdf in pdf_links:
            relative_url = pdf.get_attribute("href")
            full_url = urljoin(base_url, relative_url)
            mission_data["pdfs"].append(full_url)
         
         # Store extracted data
        mission_data_store[mission] = mission_data
    
    except Exception as e:
        print(f"Error: Element not found for {mission}: {e}")

    time.sleep(2)
    
    
    # Extract data from subpages
    for subpage in mission_subpages:
        if f"{mission}-{subpage}" == "saral-altika-references":
            subpage_url = f"{base_url}/saral-references"
        elif f"{mission}-{subpage}" == "oceansat-3-references":
            subpage_url = f"{base_url}/oceansat3-references"
        else:
            subpage_url = f"{base_url}/{mission}-{subpage}"
        driver.get(subpage_url)

        try:
            # Wait for the content area to load
            content_area = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".field-item.even"))
            )
            print(f"Element found for: {mission}-{subpage}")

            # Extract data from the subpage
            subpage_data = {
                "text": content_area.text,
                "images": [],
                "tables": [],
                "pdfs": []
            }

            # Extract images
            images = content_area.find_elements(By.TAG_NAME, "img")
            for img in images:
                img_src = urljoin(base_url, img.get_attribute("src"))
                if img_src != pdf_icon_url:
                    mission_data["images"].append(img_src)
                    
            # Extract tables
            tables = content_area.find_elements(By.TAG_NAME, "table")
            for table in tables:
                subpage_data["tables"].append(table.get_attribute("outerHTML"))

            # Extract PDF links
            pdf_links = content_area.find_elements(By.CSS_SELECTOR, "p a[href$='.pdf']")
            for pdf in pdf_links:
                relative_url = pdf.get_attribute("href")
                full_url = urljoin(base_url, relative_url)
                subpage_data["pdfs"].append(full_url)

            # Store extracted data
            mission_data_store[f"{mission}-{subpage}"] = subpage_data

        except Exception as e:
            print(f"Error: Element not found for {mission}-{subpage}: {e}")

        time.sleep(2)
        
    # Data product filed for each mission
    data_product_url = f"{base_url}/catalog/satellite.php?mission={mission.upper()}"
    driver.get(data_product_url)
    
    try:
        # Wait for dropdowns
        wait = WebDriverWait(driver, 10)
        satellite_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "satellite")))
        satellite_select = Select(satellite_dropdown)

        sensor_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "sensor")))
        sensor_select = Select(sensor_dropdown)

        data_products = []
        
        for satellite_option in satellite_select.options:
            satellite_name = satellite_option.text.strip()
            satellite_option.click()
            time.sleep(2)

            for sensor_option in sensor_select.options:
                sensor_name = sensor_option.text.strip()
                sensor_option.click()
                time.sleep(3)

                # Locate table
                table = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="tabledata"]')))
                headers = table.find_elements(By.TAG_NAME, "th")
                extracted_columns = [header.text.strip() for header in headers if header.text.strip()]

                # Extract table rows
                rows = table.find_elements(By.TAG_NAME, "tr")
                product_rows = []
                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    if cols:
                        product_rows.append([col.text.strip() for col in cols])

                # Store extracted data
                data_products.append({
                    "category": "Satellite Data",
                    "satellite": satellite_name,
                    "sensor": sensor_name,
                    "columns": extracted_columns,
                    "rows": product_rows
                })
                
        mission_data_store[f"{mission}-data-products"] = data_products
        
    except Exception as e:
        print(f"Error: Element not found for {mission}-data-products: {e}")
        
    time.sleep(2)
    
# Close the browser
driver.quit()

Element found for: insat-3d
Element found for: insat-3d-introduction
Element found for: insat-3d-objectives
Element found for: insat-3d-spacecraft
Element found for: insat-3d-payloads
Element found for: insat-3d-references
Element found for: insat-3dr
Element found for: insat-3dr-introduction
Element found for: insat-3dr-objectives
Element found for: insat-3dr-spacecraft
Element found for: insat-3dr-payloads
Element found for: insat-3dr-references
Element found for: megha-tropiques
Element found for: megha-tropiques-introduction
Element found for: megha-tropiques-objectives
Element found for: megha-tropiques-spacecraft
Element found for: megha-tropiques-payloads
Element found for: megha-tropiques-references
Error: Element not found for megha-tropiques-data-products: Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=134.0.6998.88)
Stacktrace:
	GetHandleVerifier [0x007FB5A3+24387]
	(No symbol) [0x00785904]
	(No symbol) 

In [20]:
with open("mosdac_mission_data.json", "w", encoding="utf-8") as json_file:
    json.dump(mission_data_store, json_file, ensure_ascii=False, indent=4)

print("\n✅ Data extraction completed! Saved as 'mosdac_mission_data.json'.")


✅ Data extraction completed! Saved as 'mosdac_mission_data.json'.


In [13]:
driver = webdriver.Chrome()

catalog_pages = {
    "satellite": "https://www.mosdac.gov.in/catalog/satellite.php",
    "in-situ": "https://www.mosdac.gov.in/catalog/insitu.php",
    "radar": "https://www.mosdac.gov.in/internal/catalog-radar"
}

wait = WebDriverWait(driver, 10)

data_list = []

for category, url in catalog_pages.items():
    driver.get(url)
    time.sleep(2)

# Locate the Satellite dropdown
    satellite_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "satellite")))  # Update ID
    satellite_select = Select(satellite_dropdown)

    # Locate the Sensor dropdown
    sensor_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "sensor")))  # Update ID
    sensor_select = Select(sensor_dropdown)

    # Loop through each Satellite option
    for satellite_option in satellite_select.options:
        satellite_name = satellite_option.text.strip()
        satellite_option.click()
        time.sleep(2)  # Wait for the Sensor dropdown to update

        # Loop through each Sensor option
        for sensor_option in sensor_select.options:
            sensor_name = sensor_option.text.strip()
            sensor_option.click()
            time.sleep(2)  # Wait for table to update

            # Locate table and extract rows
            table = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="tabledata"]')))  # Update ID
            rows = table.find_elements(By.TAG_NAME, "tr")

            # Extract data from table
            for row in rows:
                cols = row.find_elements(By.TAG_NAME, "td")
                if cols:
                    row_data = [col.text.strip() for col in cols]
                    row_data.insert(0, sensor_name)
                    row_data.insert(0, satellite_name)
                    row_data.insert(0, category)
                    data_list.append(row_data)

            print(f"Scraped: {category} > {satellite_name} > {sensor_name}")

import json

# Determine the maximum number of columns in data_list
max_cols = max(len(row) for row in data_list)
print(f"Max columns in data_list: {max_cols}")

# Define column names dynamically
# column_names = ["Category", "Satellite", "Sensor", ] + [f"Col_{i}" for i in range(max_cols - 3)]
base_column_names = ["Category", "Satellite", "Sensor", "Sr.No", "Product/Acq Calendar/Version", "Product Description", 
                "Processing Level", "Temporal Resolution", "Start Date", "End Date", "Processing Status", "DOI"]

if len(base_column_names) < max_cols:
    additional_cols = [f"Extra_Col_{i}" for i in range(max_cols - len(base_column_names))]
    column_names = base_column_names + additional_cols
else:
    column_names = base_column_names[:max_cols]  # Trim extra column names if not needed

# Pad shorter rows with empty values to match the max column length
normalized_data = [row + [""] * (max_cols - len(row)) for row in data_list]

# Convert to DataFrame
df = pd.DataFrame(normalized_data, columns=column_names)

# Convert to JSON
json_data = df.to_dict(orient="records")

# # Save JSON file
# with open("mosdac_data.json", "w", encoding="utf-8") as f:
#     json.dump(json_data, f, indent=4)


# Close driver
driver.quit()
# print("Scraping Complete! Data saved to mosdac_data.json")  

Scraped: satellite > EOS-06 > OCM
Scraped: satellite > EOS-06 > SCATTEROMETER
Scraped: satellite > INSAT-3DR > IMAGER
Scraped: satellite > INSAT-3DR > SOUNDER
Scraped: satellite > INSAT-3DS > IMAGER
Scraped: satellite > INSAT-3DS > SOUNDER
Scraped: satellite > SARAL > ALTIMETER
Scraped: satellite > EOS-07 > MHS
Scraped: satellite > INSAT-3A > CCD
Scraped: satellite > INSAT-3A > VHRR
Scraped: satellite > INSAT-3D > IMAGER
Scraped: satellite > INSAT-3D > SOUNDER
Scraped: satellite > KALPANA-1 > VHRR
Scraped: satellite > MEGHATROPIQUES > MADRAS
Scraped: satellite > MEGHATROPIQUES > ROSA
Scraped: satellite > MEGHATROPIQUES > SAPHIR
Scraped: satellite > MEGHATROPIQUES > SCARAB
Scraped: satellite > OCEANSAT-2 > SCATTEROMETER
Scraped: satellite > SCATSAT-1 > SCATTEROMETER
Scraped: in-situ > In-situ > AMS
Scraped: in-situ > In-situ > AWS
Scraped: in-situ > In-situ > AWSAGRI
Scraped: in-situ > In-situ > AWSUPG
Scraped: in-situ > In-situ > IMDAWS
Scraped: radar > RADAR > S BAND DWR CHERRAPUNJI
S

In [ ]:
# Remove empty columns from JSON output
json_data = []
for row in df.to_dict(orient="records"):
    cleaned_row = {key: value for key, value in row.items() if value != ""}
    json_data.append(cleaned_row)

# Save JSON file
with open("mosdac_catalog_data.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=4)

In [17]:
driver = webdriver.Chrome()

catalog_pages = {
    "satellite": "https://www.mosdac.gov.in/catalog/satellite.php",
    "in-situ": "https://www.mosdac.gov.in/catalog/insitu.php",
    "radar": "https://www.mosdac.gov.in/internal/catalog-radar"
}

wait = WebDriverWait(driver, 10)
data_list = []
column_names = ["Category", "Satellite", "Sensor"]  # Fixed Columns

for category, url in catalog_pages.items():
    driver.get(url)
    time.sleep(3)

    # Locate dropdowns
    satellite_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "satellite")))
    satellite_select = Select(satellite_dropdown)

    sensor_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "sensor")))
    sensor_select = Select(sensor_dropdown)

    # Loop through each Satellite
    for satellite_option in satellite_select.options:
        satellite_name = satellite_option.text.strip()
        satellite_option.click()
        time.sleep(2)

        # Loop through each Sensor
        for sensor_option in sensor_select.options:
            sensor_name = sensor_option.text.strip()
            sensor_option.click()
            time.sleep(3)

            # Locate table and extract headers dynamically
            table = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="tabledata"]')))
            headers = table.find_elements(By.TAG_NAME, "th")

            # Extract column names dynamically (skip empty headers)
            extracted_columns = [header.text.strip() for header in headers if header.text.strip()]
            
            # If first time, extend column_names list
            if len(column_names) == 3:  
                column_names.extend(extracted_columns)

            # Extract table rows
            rows = table.find_elements(By.TAG_NAME, "tr")
            for row in rows:
                cols = row.find_elements(By.TAG_NAME, "td")
                if cols:
                    row_data = [col.text.strip() for col in cols]
                    row_data = [category, satellite_name, sensor_name] + row_data
                    data_list.append(row_data)

            print(f"Scraped: {category} > {satellite_name} > {sensor_name}")

# Find the maximum number of columns from data
max_cols = max(len(row) for row in data_list)

# Adjust column_names dynamically based on extracted data
if len(column_names) < max_cols:
    extra_cols = [f"Col_{i}" for i in range(len(column_names), max_cols)]
    column_names.extend(extra_cols)
elif len(column_names) > max_cols:
    column_names = column_names[:max_cols]  # Trim extra columns

# Normalize Data: Pad shorter rows with empty values
normalized_data = [row + [""] * (max_cols - len(row)) for row in data_list]

# Convert to DataFrame
df = pd.DataFrame(normalized_data, columns=column_names)

# Convert to JSON
json_data = df.to_dict(orient="records")

# Close driver
driver.quit()

print("Scraping Complete!")

Scraped: satellite > EOS-06 > OCM
Scraped: satellite > EOS-06 > SCATTEROMETER
Scraped: satellite > INSAT-3DR > IMAGER
Scraped: satellite > INSAT-3DR > SOUNDER
Scraped: satellite > INSAT-3DS > IMAGER
Scraped: satellite > INSAT-3DS > SOUNDER
Scraped: satellite > SARAL > ALTIMETER
Scraped: satellite > EOS-07 > MHS
Scraped: satellite > INSAT-3A > CCD
Scraped: satellite > INSAT-3A > VHRR
Scraped: satellite > INSAT-3D > IMAGER
Scraped: satellite > INSAT-3D > SOUNDER
Scraped: satellite > KALPANA-1 > VHRR
Scraped: satellite > MEGHATROPIQUES > MADRAS
Scraped: satellite > MEGHATROPIQUES > ROSA
Scraped: satellite > MEGHATROPIQUES > SAPHIR
Scraped: satellite > MEGHATROPIQUES > SCARAB
Scraped: satellite > OCEANSAT-2 > SCATTEROMETER
Scraped: satellite > SCATSAT-1 > SCATTEROMETER
Scraped: in-situ > In-situ > AMS
Scraped: in-situ > In-situ > AWS
Scraped: in-situ > In-situ > AWSAGRI
Scraped: in-situ > In-situ > AWSUPG
Scraped: in-situ > In-situ > IMDAWS
Scraped: radar > RADAR > S BAND DWR CHERRAPUNJI
S

In [18]:
with open("mosdac_catalog_data_v2.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=4)

print("Data saved to 'mosdac_catalog_data_v2.json'.")    

Data saved to 'mosdac_catalog_data_v2.json'.
